# PV segmentation

Comparison for finloop - YOLO-segment vs. YOLOdetect + SAM

source (Huggingface): finloop/yolov8s-seg-solar-panels (aka Rzeszów model)

credits: https://blog.roboflow.com/how-to-use-yolov8-with-sam/ (Roboflow)

XI 25

*MD*

note: a dual-use model (detection + instance segmentation)

requires OBB-versioned datasets

## libs

In [1]:
# %pip install numpy
# %pip install pandas
# %pip install ultralytics

In [2]:
import matplotlib.pyplot as plt
import cv2
from ultralytics import YOLO, SAM
import torch

In [3]:
dev = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
dev

device(type='cuda', index=0)

## defs

### data

In [4]:
pilot = 'pilotPV_panels.v1i.yolov8-obb/test/images/*.jpg'
rzeszow_test = 'rzeszowSolar panels seg.v2i.yolov8-obb/test/images/*.jpg'
rzeszow_train = 'rzeszowSolar panels seg.v2i.yolov8-obb/train/images/*.jpg'
rzeszow_valid = 'rzeszowSolar panels seg.v2i.yolov8-obb/valid/images/*.jpg'
synth_test = 'auto_pv_to_fine_tunning.v4i.yolov8-obb/test/images/*.jpg'
synth_train = 'auto_pv_to_fine_tunning.v4i.yolov8-obb/train/images/*.jpg'
synth_valid = 'auto_pv_to_fine_tunning.v4i.yolov8-obb/valid/images/*.jpg'

In [5]:
proste_1_zdj = "rzeszowSolar panels seg.v2i.yolov8-obb/test/images/022_jpg.rf.f76ab3a091f7c7931d2c26cc7375842b.jpg"
data_p = proste_1_zdj
pth = proste_1_zdj
nazwa = "proste_1_zdj"

### model

In [6]:
model_pt = "best.pt"

In [7]:
model = YOLO(model=model_pt, task="detect", verbose=True)

In [8]:
sam = SAM("sam_b.pt")

### segment analysis

In [9]:
def sum_pv_segments(pth, nazwa="no_info_run", model=model, print_info=False, disp_img=False, display_coef=100):
    pv_area = 0
    yolo_results = model(pth, save=print_info, name=nazwa, stream=True, device=dev, verbose=print_info, retina_masks=True)
    for i, res in enumerate(yolo_results):
        dsp = disp_img and i % display_coef == 0 # limit
        ppth = res.path # nicely conveyed
        img_w, img_h = res.orig_shape # although YOLO reshapes img when necessary, retina_masks=True makes masks match org img
        if dsp:
            image = cv2.cvtColor(cv2.imread(ppth), cv2.COLOR_BGR2RGB)
            image = torch.tensor(image, device=dev)
        if res is not None and res.masks is not None:
            binary_mask = torch.where(res.masks.data == True, 1, 0)
            mask_sum = binary_mask.sum(axis=0).data
            mask_sum_damped = torch.where(mask_sum >= 1, 1, 0) # actually some pxs covered multiple times - same object
            if dsp:
                bcg_white = torch.ones_like(image)*255
                new_image = bcg_white * (1 - mask_sum_damped[..., torch.newaxis]) + image * mask_sum_damped[..., torch.newaxis]
                plt.imshow(new_image.reshape((img_w, img_h, 3)).cpu())
                plt.title(f"Masked PVs in {ppth[ppth.rfind('/'):]}")
                plt.axis('off')
                plt.show()
                # cv2.imwrite('c.png', new_image.reshape((img_w, img_h, 3)).cpu().numpy())
            pv_area += mask_sum_damped.sum().div(img_w*img_h) # percentage
            # print('mask sums', mask_sum.sum(), mask_sum_damped.sum())
            if print_info:
                print(i, pv_area.item())
        if dsp:
            plt.imshow(image.cpu())
            plt.title(f"base img {ppth[ppth.rfind('/'):]}")
            plt.axis('off')
            plt.show()
    return pv_area

In [10]:
def sum_pv_segments_sam(pth, nazwa="no_info_run", model=model, print_info=False, disp_img=False, display_coef=100):
    pv_area = 0
    yolo_results = model(pth, save=print_info, name=nazwa, stream=True, device=dev, verbose=print_info)
    for i, res in enumerate(yolo_results):
        dsp = disp_img and i % display_coef == 0 # limit
        ppth = res.path # nicely conveyed
        img_w, img_h = res.orig_shape # although YOLO reshapes img when necessary, SAM masks match org img
        if dsp:
            image = cv2.cvtColor(cv2.imread(ppth), cv2.COLOR_BGR2RGB)
            image = torch.tensor(image, device=dev)
        if res is not None and res.boxes is not None and res.boxes.xyxy is not None and len(res.boxes.xyxy > 0): # null-len res.boxes.xyxy for no PV
            sam_results = sam.predict(source=ppth, bboxes=res.boxes.xyxy)
            if sam_results is not None and sam_results[0] is not None and sam_results[0].masks is not None and sam_results[0].masks.data is not None:
                binary_mask = torch.where(sam_results[0].masks.data == True, 1, 0)
                mask_sum = binary_mask.sum(axis=0).data
                mask_sum_damped = torch.where(mask_sum >= 1, 1, 0) # actually some pxs covered multiple times - same object
                if dsp:
                    bcg_white = torch.ones_like(image)*255
                    new_image = bcg_white * (1 - mask_sum_damped[..., torch.newaxis]) + image * mask_sum_damped[..., torch.newaxis]
                    plt.imshow(new_image.reshape((img_w, img_h, 3)).cpu())
                    plt.title(f"Masked PVs in {ppth[ppth.rfind('/'):]}")
                    plt.axis('off')
                    plt.show()
                    # cv2.imwrite('c.png', new_image.reshape((img_w, img_h, 3)).cpu().numpy())
                pv_area += mask_sum_damped.sum().div(img_w*img_h) # percentage
                # print('mask sums', mask_sum.sum(), mask_sum_damped.sum())
                if print_info:
                    print(i, pv_area.item())
        if dsp:
            plt.imshow(image.cpu())
            plt.title(f"base img {ppth[ppth.rfind('/'):]}")
            plt.axis('off')
            plt.show()
    return pv_area

## pilot

In [11]:
sum_pv_segments(pilot, "pilot_segment")

tensor(0.2908, device='cuda:0')

In [12]:
sum_pv_segments_sam(pilot, "pilot_segment")


image 1/1 /home/marekd6/ZPB/modelling/modelling/pilotPV_panels.v1i.yolov8-obb/test/images/Zrzut-ekranu-291-_png.rf.c105b47207c9dc4203900bab38f73c89.jpg: 1024x1024 1 0, 1 1, 1 2, 7037.6ms
Speed: 41.1ms preprocess, 7037.6ms inference, 3.9ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /home/marekd6/ZPB/modelling/modelling/pilotPV_panels.v1i.yolov8-obb/test/images/Zrzut-ekranu-293-_png.rf.6501de77787921a2b0c0a3fac3ff7882.jpg: 1024x1024 1 0, 1 1, 1 2, 1 3, 1 4, 1 5, 1 6, 1 7, 1 8, 1 9, 1 10, 1 11, 1 12, 1146.7ms
Speed: 9.4ms preprocess, 1146.7ms inference, 1.9ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /home/marekd6/ZPB/modelling/modelling/pilotPV_panels.v1i.yolov8-obb/test/images/Zrzut-ekranu-294-_png.rf.ec19d20eeaa7c3a1ea9c829158bbebdc.jpg: 1024x1024 1 0, 1 1, 1 2, 1 3, 1064.7ms
Speed: 6.0ms preprocess, 1064.7ms inference, 1.3ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /home/marekd6/ZPB/modelling/modelling/pilotPV_panels.v1i.yolov8

tensor(0.2995, device='cuda:0')

## synthetic

### train

In [13]:
sum_pv_segments(synth_train)

tensor(3.0919, device='cuda:0')

In [14]:
sum_pv_segments_sam(synth_train)


image 1/1 /home/marekd6/ZPB/modelling/modelling/auto_pv_to_fine_tunning.v4i.yolov8-obb/train/images/Screenshot-2025-05-05-141951_png.rf.71f8969cded34224572d3d3b4b7bbf07.jpg: 1024x1024 1 0, 1050.7ms
Speed: 4.4ms preprocess, 1050.7ms inference, 0.7ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /home/marekd6/ZPB/modelling/modelling/auto_pv_to_fine_tunning.v4i.yolov8-obb/train/images/Screenshot-2025-05-05-141951_png.rf.e8fc3301803b30373229d81dfd39a192.jpg: 1024x1024 1 0, 1 1, 1053.5ms
Speed: 3.9ms preprocess, 1053.5ms inference, 1.0ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /home/marekd6/ZPB/modelling/modelling/auto_pv_to_fine_tunning.v4i.yolov8-obb/train/images/Screenshot-2025-05-05-141951_png.rf.fce413ee410f6a8aa17e7b99dccc1725.jpg: 1024x1024 1 0, 1056.6ms
Speed: 4.3ms preprocess, 1056.6ms inference, 1.1ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /home/marekd6/ZPB/modelling/modelling/auto_pv_to_fine_tunning.v4i.yolov8-obb/train/i

tensor(3.1544, device='cuda:0')

### val

In [15]:
sum_pv_segments(synth_valid)

tensor(0.3521, device='cuda:0')

In [16]:
sum_pv_segments_sam(synth_valid)


image 1/1 /home/marekd6/ZPB/modelling/modelling/auto_pv_to_fine_tunning.v4i.yolov8-obb/valid/images/Screenshot-2025-05-05-142243_png.rf.0c1fc1b110982ed45002e385f2ed0cdf.jpg: 1024x1024 1 0, 1 1, 1100.0ms
Speed: 5.0ms preprocess, 1100.0ms inference, 1.7ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /home/marekd6/ZPB/modelling/modelling/auto_pv_to_fine_tunning.v4i.yolov8-obb/valid/images/Screenshot-2025-05-05-142342_png.rf.d7373a2eee092999859ee3a20a9e7b13.jpg: 1024x1024 1 0, 1 1, 1 2, 1 3, 1 4, 1 5, 1 6, 1 7, 1 8, 1 9, 1 10, 1 11, 1 12, 1 13, 1 14, 1 15, 1 16, 1 17, 1214.1ms
Speed: 3.6ms preprocess, 1214.1ms inference, 1.6ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /home/marekd6/ZPB/modelling/modelling/auto_pv_to_fine_tunning.v4i.yolov8-obb/valid/images/Screenshot-2025-05-05-142412_png.rf.7346005e071261338a10472ee5ae78b1.jpg: 1024x1024 1 0, 1 1, 1 2, 1 3, 1 4, 1 5, 2060.2ms
Speed: 3.9ms preprocess, 2060.2ms inference, 0.8ms postprocess per image at sh

tensor(0.3478, device='cuda:0')

### test

In [17]:
sum_pv_segments(synth_test)

tensor(0.3837, device='cuda:0')

In [18]:
sum_pv_segments_sam(synth_test)


image 1/1 /home/marekd6/ZPB/modelling/modelling/auto_pv_to_fine_tunning.v4i.yolov8-obb/test/images/Screenshot-2025-05-05-143003_png.rf.fdf19d2ceff2192c26f9132617f7138b.jpg: 1024x1024 1 0, 1 1, 1 2, 1 3, 1127.5ms
Speed: 4.8ms preprocess, 1127.5ms inference, 0.9ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /home/marekd6/ZPB/modelling/modelling/auto_pv_to_fine_tunning.v4i.yolov8-obb/test/images/Screenshot-2025-05-05-143259_png.rf.5e77841caaec3af9f76d66e402dada53.jpg: 1024x1024 1 0, 1 1, 1 2, 1 3, 1102.3ms
Speed: 5.4ms preprocess, 1102.3ms inference, 1.8ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /home/marekd6/ZPB/modelling/modelling/auto_pv_to_fine_tunning.v4i.yolov8-obb/test/images/Screenshot-2025-05-05-143340_png.rf.a3a5eb36c1ae28cb99b95aa17e3d2dd7.jpg: 1024x1024 1 0, 1 1, 1 2, 1 3, 1065.8ms
Speed: 3.7ms preprocess, 1065.8ms inference, 1.4ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /home/marekd6/ZPB/modelling/modelling/auto_pv_t

tensor(0.3841, device='cuda:0')

## Rzeszów

### train

In [ ]:
# sum_pv_segments(rzeszow_train)

In [ ]:
# sum_pv_segments_sam(rzeszow_train)

### val

In [ ]:
sum_pv_segments(rzeszow_valid)

In [ ]:
sum_pv_segments_sam(rzeszow_valid)

### test

In [ ]:
sum_pv_segments(rzeszow_test)

In [ ]:
sum_pv_segments_sam(rzeszow_test)